Notebook for optimizing the trend analyzer code.
To run the notebook, run the command "jupyter notebook --ip=0.0.0.0 --port=5000 --allow-root --no-browser". Then open the webpage that opens in replit in a new tab, and enter the token for the server you find from the command "jupter server list". If you enter the token in the replit preview it will give you a 403 error.

In [ ]:
#import necessary modules
from modules import *
import os
import mlflow

In [ ]:
#setup mlflow
mlflow_tracking_uri = "../../mlflow/experiments"
mlflow.set_tracking_uri(mlflow_tracking_uri)
mlflow.set_experiment("TrendFinderOptimizer")
mlflow.dspy.autolog()

In [ ]:
#setup dspy
api_key = os.environ['paul2']
lm = dspy.LM('gemini/gemini-2.5-flash', api_key=api_key, max_tokens=8000)
dspy.configure(lm=lm)

## DSPy Optimization for doc_analyzer

Since we don't have labeled training data, we'll use DSPy's signature optimization and prompt engineering techniques to improve the `doc_analyzer` signature performance.

In [ ]:
# Import optimization modules
import dspy
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate import Evaluate
import json
import pandas as pd

In [ ]:
# Create synthetic examples for different document types and categories
# This helps DSPy understand the expected input/output patterns

def create_synthetic_examples():
    examples = []
    
    # Example 1: Realistic inspection report with lengthy content
    inspection_report_1 = """
    INSPECTION REPORT - Document ID: 32037 ALR10C6BZ 097
    Date: 07-10-2025
    Inspector: AEPACS
    Location: Industrial Site NA

    SUMMARY OF INSPECTION:
    This inspection was conducted in accordance with safety regulations and compliance standards.
    The facility was evaluated for adherence to safety protocols, equipment maintenance, and regulatory compliance.

    VIOLATIONS IDENTIFIED:
    1. Missing safety equipment in Zone A - Safety harnesses not available at designated stations
    2. Improper chemical storage in Building 3 - Hazardous materials not properly labeled or contained
    3. Emergency exit blocked by equipment in Warehouse B

    RECOMMENDATIONS:
    - Immediate procurement and installation of safety equipment
    - Proper labeling and containment of all hazardous materials
    - Clear all emergency exits of obstructions

    COMPLIANCE STATUS: Non-compliant - 3 violations found
    Next inspection scheduled for 08-15-2025
    """

    examples.append(dspy.Example(
        document=inspection_report_1,
        categories=["document", "number of violations", "list and details of violations"],
        in_csv="",
        last_context="Analyzing safety inspection reports for compliance trends"
    ).with_inputs("document", "categories", "in_csv", "last_context"))

    # Example 2: Financial document with some missing categories
    financial_report = """
    QUARTERLY FINANCIAL STATEMENT - Q3 2025
    Company: TechCorp Industries

    REVENUE BREAKDOWN:
    Product Sales: $850,000
    Service Revenue: $320,000
    Total Revenue: $1,170,000

    OPERATIONAL EXPENSES:
    Salaries and Benefits: $450,000
    Equipment and Maintenance: $120,000
    Marketing: $85,000
    Total Expenses: $655,000

    NET PROFIT: $515,000

    Note: Employee count data not available in this quarterly report.
    Tax information will be provided in annual filing.
    """

    examples.append(dspy.Example(
        document=financial_report,
        categories=["revenue", "expenses", "profit", "employee_count", "tax_rate"],
        in_csv="",
        last_context="Processing quarterly financial reports for trend analysis"
    ).with_inputs("document", "categories", "in_csv", "last_context"))

    # Example 3: Document with no violations (testing N/A handling)
    clean_inspection = """
    INSPECTION REPORT - Document ID: 45892 CLN001
    Date: 07-12-2025
    Inspector: SAFETY_TEAM_A
    Location: Corporate Headquarters

    INSPECTION SUMMARY:
    Comprehensive safety and compliance inspection conducted across all floors and departments.
    All safety protocols, equipment, and procedures were found to be in full compliance.

    FINDINGS:
    - All safety equipment properly maintained and accessible
    - Emergency exits clear and properly marked
    - Chemical storage in full compliance with regulations
    - Fire safety systems operational and up to date

    VIOLATIONS: None identified
    COMPLIANCE STATUS: Fully compliant
    Commendation for excellent safety standards maintained.
    """

    examples.append(dspy.Example(
        document=clean_inspection,
        categories=["document", "number of violations", "list and details of violations"],
        in_csv="document,number of violations,list and details of violations\n32037_ALR10C6BZ_097.pdf,3,Missing safety equipment in Zone A; Improper chemical storage in Building 3; Emergency exit blocked in Warehouse B",
        last_context="Safety equipment and storage violations are recurring issues across sites"
    ).with_inputs("document", "categories", "in_csv", "last_context"))
    
    return examples

synthetic_examples = create_synthetic_examples()
print(f"Created {len(synthetic_examples)} synthetic examples")

In [ ]:
# Define evaluation metrics for the doc_analyzer
def evaluate_doc_analyzer_output(example, pred, trace=None):
    """Evaluate the quality of doc_analyzer output"""
    score = 0.0
    max_score = 5.0

    # 1. Check if CSV format is valid
    try:
        import io
        pd.read_csv(io.StringIO(pred.out_csv))
        score += 1.0  # Valid CSV format
    except:
        pass

    # 2. Check if all categories are addressed in CSV headers
    try:
        csv_data = pd.read_csv(io.StringIO(pred.out_csv))
        csv_columns = set(csv_data.columns.tolist())
        expected_categories = set(example.categories)
        if expected_categories.issubset(csv_columns):
            score += 1.0
    except:
        pass

    # 3. Context management - penalize excessive length, reward meaningful updates
    context_score = 0.0
    if pred.next_context != example.last_context:
        # Reward context updates but penalize excessive length
        context_length = len(pred.next_context.split())
        if 5 <= context_length <= 50:  # Reasonable context length
            context_score = 1.0
        elif context_length > 50:  # Too long - partial credit
            context_score = 0.5
        elif context_length > 0:  # Very short but present
            context_score = 0.3
    score += context_score

    # 4. Appropriate use of N/A - check if N/A is used reasonably
    try:
        csv_data = pd.read_csv(io.StringIO(pred.out_csv))
        last_row = csv_data.iloc[-1]  # Get the newly added row

        # Count N/A values in the new row
        na_count = sum(1 for val in last_row if str(val).strip().upper() == 'N/A')
        total_categories = len(example.categories)

        # Score based on appropriate N/A usage
        if na_count == 0:  # No N/A - good if data is available
            score += 1.0
        elif na_count < total_categories:  # Some N/A - partial data extracted
            score += 0.8
        elif na_count == total_categories:  # All N/A - only if truly no data
            # This should be rare and only for documents with no extractable data
            score += 0.3
    except:
        pass

    # 5. Data extraction quality - check if actual data was extracted when possible
    try:
        csv_data = pd.read_csv(io.StringIO(pred.out_csv))
        last_row = csv_data.iloc[-1]

        # Check if meaningful data was extracted (not just filename)
        meaningful_data = False
        for col in csv_data.columns:
            if col != 'document':  # Skip document name column
                val = str(last_row[col]).strip()
                if val not in ['N/A', '', 'nan'] and len(val) > 1:
                    meaningful_data = True
                    break

        if meaningful_data:
            score += 1.0
        else:
            score += 0.2  # Some credit for proper format even without data
    except:
        pass

    return score / max_score

print("Evaluation metric defined")

## Note about DSPy Example Format

The error we encountered earlier was due to improper example formatting. The `with_inputs()` method is crucial for DSPy optimization to work properly. It tells DSPy which fields are inputs vs outputs for the signature.

In [ ]:
# Alternative approach: Manual prompt optimization
# Since BootstrapFewShot requires proper examples, let's try a simpler approach

print("Setting up manual optimization approach...")
print("This approach focuses on improving the signature description and few-shot examples.")

# Create an optimized version of the doc_analyzer signature
class optimized_doc_analyzer(dspy.Signature):
    """Advanced document analyzer that extracts structured information from documents and maintains CSV format.
    
    Key instructions:
    1. Extract data for each category from the document content
    2. Use 'N/A' when information is not available or not applicable
    3. Maintain proper CSV format with headers and comma separation
    4. Keep context updates brief and relevant to avoid token bloat
    5. Focus on extracting meaningful data patterns and trends"""
    
    document: Attachments = dspy.InputField(
        desc="Document to analyze - extract information matching the specified categories")
    categories: list[str] = dspy.InputField(
        desc="List of data categories to extract from the document (e.g., 'violations', 'revenue', etc.)")
    in_csv: str = dspy.InputField(
        desc="Existing CSV data with headers. Append new row with current document's data")
    last_context: str = dspy.InputField(
        desc="Previous context and trends identified. Keep this information and add new insights if significant")
    
    next_context: str = dspy.OutputField(
        desc="Updated context with new trends or patterns. Keep brief (under 50 words) to prevent context bloat")
    out_csv: str = dspy.OutputField(
        desc="CSV with new row added. Format: proper headers, comma-separated values, use 'N/A' for missing data")

print("Optimized signature created with enhanced instructions")

In [ ]:
# Create the optimized module and save it
optimized_module = dspy.ChainOfThought(optimized_doc_analyzer)

# Test the optimized module with a simple example
test_doc = "Sample inspection report with 2 violations: missing fire extinguisher and blocked exit."
test_result = optimized_module(
    document=test_doc,
    categories=["document", "number of violations"],
    in_csv="document,number of violations",
    last_context="Testing optimized signature"
)

print("Test Results:")
print(f"Context: {test_result.next_context}")
print(f"CSV Output: {test_result.out_csv}")

# Create directory for optimized modules
import os
os.makedirs('optimized_modules', exist_ok=True)

# Save the optimized module
optimized_module.save('optimized_modules/optimized_doc_analyzer.json')
print("\nOptimized module saved to 'optimized_modules/optimized_doc_analyzer.json'")

## How to Use the Optimized Module

The optimized signature has been saved and can be used in your main code. Here are two ways to integrate it:

In [ ]:
# Option 1: Load and use directly
import dspy
from modules import doc_analyzer

# Create and load optimized module
optimized_doc_analyzer_module = dspy.ChainOfThought(optimized_doc_analyzer)
# Note: The optimized version uses the enhanced signature defined above

# Option 2: Modify your trend_analyzer class
class optimized_trend_analyzer(dspy.Module):
    def __init__(self, use_optimized=True):
        super().__init__()
        if use_optimized:
            self.doc_analyzer_sql = dspy.ChainOfThought(optimized_doc_analyzer)
            print("Using optimized doc_analyzer signature")
        else:
            from modules import doc_analyzer
            self.doc_analyzer_sql = dspy.ChainOfThought(doc_analyzer)
            print("Using original doc_analyzer signature")

    def forward(self, documents: list[Attachments], categories: list[str], context: str):
        # Same implementation as before
        doc_summary = ""
        for document in documents:
            result = self.doc_analyzer_sql(
                document=document,
                categories=categories,
                in_csv=doc_summary,
                last_context=context
            )
            context = result.next_context
            doc_summary = result.out_csv
        return doc_summary, context

print("Usage examples defined. You can now use optimized_trend_analyzer in your main code.")